In [ ]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
from copy import copy
from functools import partial
from typing import Dict
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from matplotlib.ticker import MultipleLocator

from main.utils.data import split_dataset, raw_data_preparation
from main.utils.utils import optimization_process
from main.settings import train_validation_config

# 365 Days Average

In [ ]:
week = 52
n_gap = 7
n_out = 4
scale = 100000

n_splits = train_validation_config['n_splits']
max_train_size = train_validation_config['max_train_size']

df = raw_data_preparation()

daily_data = df[['A']]
daily_data.fillna(0, inplace=True)

tscv = TimeSeriesSplit(n_splits=n_splits + (n_out - 1), test_size=7, max_train_size=max_train_size)

predictions_365 = []
observations = []

for train_index, val_index in tscv.split(daily_data):
    if (train_index[-1] + n_out * 7 + 1) <= len(daily_data):
        val_index = np.arange(train_index[-1] + 1, train_index[-1] + n_out * 7 + 1)
    
        train = daily_data.iloc[train_index] / scale
        val = daily_data.iloc[val_index] / scale

        yhat = [train[-(n_gap * 7 + 365):-(n_gap * 7)].values.mean() * 7] * n_out
        y = np.array(np.split(val.values.reshape(-1,), n_out)).sum(axis=1)

        predictions_365.append(yhat)
        observations.append(y)
    
predictions_365 = np.array(predictions_365)
observations = np.array(observations)

In [ ]:
print(f'mse = {mean_squared_error(observations, predictions_365)}')

diff = observations - predictions_365
diff_square = np.power(diff, 2)

baseline_365 = np.sqrt(diff_square.mean(axis=1))

# SARIMA Bench Mark

In [ ]:
df = raw_data_preparation()
df['week'] = [idx.week for idx in df.index]
df['year'] = [idx.year for idx in df.index]

df_A_year_week = df.groupby(['week', 'year'], as_index=False)['A'].sum()

df_A_year_week.sort_values(by=['year', 'week'], inplace=True)

df_A_year_week.reset_index(drop=True, inplace=True)

A_week = df_A_year_week['A'].values


def sarima_moving_window(p, d, q, season_p, season_d, season_q, s):
    
    tscv = TimeSeriesSplit(n_splits=n_splits + (n_out - 1), test_size=1, max_train_size=max_train_size//7)

    predictions = []
    observations = []

    for train_index, val_index in tscv.split(A_week):

        if (train_index[-1] + n_out + 1) <= len(A_week):
            val_index = np.arange(train_index[-1] + 1, train_index[-1] + n_out + 1)

            train = A_week[train_index] / scale
            val = A_week[np.concatenate((train_index[-(n_gap):], val_index))] / scale

            sarima = SARIMAX(train[:-n_gap], order=(p,d,q), seasonal_order=(season_p, season_d, season_q, s))
            model = sarima.fit()
            p_weekly = model.forecast(steps=(n_gap + n_out))[-n_out:]
            o_weekly = A_week[val_index] / scale

            predictions.append(p_weekly)
            observations.append(o_weekly)

    predictions = np.array(predictions)
    observations = np.array(observations)
    
    return predictions, observations

- season_P = 1
- s = 11

In [ ]:
predictions_sarima_p1_s11, observations = sarima_moving_window(0, 0, 0, 1, 0, 0, 11)

In [ ]:
# fig, ax = plt.subplots(figsize=(16, 12))

# ax.plot(observations.reshape(-1, ), c='b', label='observations')
# ax.plot(predictions_sarima_p1_s11.reshape(-1, ), c='r', label='predictions')

# time = [str(d)[:10] for d in df.iloc[-week * 7:].index[::7].values]

# ax.set_title('SARIMA(P=1, S=11)', fontsize=24)
# ax.set_xlim(0, 51)
# ax.xaxis.set_major_locator(MultipleLocator(4))
# ax.set_xticklabels(labels=[0] + time[::4], rotation=45)
# ax.set_xlabel('Time', fontsize=24)
# ax.set_ylabel('A weekly consumption (100000)', fontsize=24)

# ax.legend()

In [ ]:
print(f'mse = {mean_squared_error(observations, predictions_sarima_p1_s11)}')

diff = observations - predictions_sarima_p1_s11
diff_square = np.power(diff, 2)

arima_p1_s11 = np.sqrt(diff_square.mean(axis=1))

# print(f'weekly mse = {np.sqrt(diff_square.mean(axis=0))}')

In [ ]:
arima_p1_s11.shape

- Q=1
- S=11

In [ ]:
predictions_sarima_q1_s11, observations = sarima_moving_window(p=0, d=0, q=0, season_p=0, season_d=0, season_q=1, s=11)
print(f'\nmse = {mean_squared_error(observations, predictions_sarima_q1_s11)}')
diff = observations - predictions_sarima_q1_s11
diff_square = np.power(diff, 2)
arima_q1_s11 = np.sqrt(diff_square.mean(axis=1))

- P=1
- Q=1
- S=11

In [ ]:
predictions_sarima_p1_q1_s11, observations = sarima_moving_window(p=0, d=0, q=0, season_p=1, season_d=0, season_q=1, s=11)

print(f'\nmse = {mean_squared_error(observations, predictions_sarima_p1_q1_s11)}')

diff = observations - predictions_sarima_p1_q1_s11
diff_square = np.power(diff, 2)

arima_p1_q1_s11 = np.sqrt(diff_square.mean(axis=1))

- P=1
- S=8

In [ ]:
predictions_sarima_p1_s8, observations = sarima_moving_window(p=0, d=0, q=0, season_p=1, season_d=0, season_q=0, s=8)
print(f'\nmse = {mean_squared_error(observations, predictions_sarima_p1_s8)}')
diff = observations - predictions_sarima_p1_s8
diff_square = np.power(diff, 2)
arima_p1_s8 = np.sqrt(diff_square.mean(axis=1))

- P=1
- S=4

In [ ]:
predictions_sarima_p1_s4, observations = sarima_moving_window(p=0, d=0, q=0, season_p=1, season_d=0, season_q=0, s=4)
print(f'\nmse = {mean_squared_error(observations, predictions_sarima_p1_s4)}')
diff = observations - predictions_sarima_p1_s4
diff_square = np.power(diff, 2)
arima_p1_s4 = np.sqrt(diff_square.mean(axis=1))

# 28 days to 4 weeks exploration

In [ ]:
import pandas as pd

from train.training_process import training_process
from main.utils.utils import optimization_process
from main.settings import train_validation_config, requirements

n_input = requirements['n_input']
n_out = requirements['n_out']
n_gap = requirements['n_gap']

In [ ]:
# Optimization
# from functools import partial
# from collections import OrderedDict

# '''
# 10.43

# beta_1 = 0.6799
# beta_2 = 0.7932
# decoder_dense_units = 11
# epochs = 13
# epsilon = 0.007872
# learning_rate = 0.002073
# lstm_units = 128
# '''

# model_name = 'lstm_v1'
# model_type = 'daily2weekly'

df = raw_data_preparation()

df['A_diff'] = df['A'].diff()

daily_data = df[['A', 'C', 'G', 'A_diff']]

statistical_operation = OrderedDict()

statistical_operation[0] = ['sum']
statistical_operation[1] = ['sum']
statistical_operation[2] = ['sum']
statistical_operation[3] = ['sum']

In [ ]:
# moving window prediction
from train.training_process import moving_window_predictions

observations, predictions_lstm_v1 = moving_window_predictions(daily_data, model_name='lstm_v1', model_type='daily2weekly', lstm_units=128, decoder_dense_units=11, 
                                                              epochs=13, statistical_operation=statistical_operation, learning_rate=0.002073, beta_1=0.6799, 
                                                              beta_2=0.7932, epsilon=0.007872, n_input=n_input, n_out=n_out, n_gap=n_gap)

In [ ]:
print(f'mse = {mean_squared_error(observations, predictions_lstm_v1)}')

diff = observations - predictions_lstm_v1
diff_square = np.power(diff, 2)

lstm_v1 = np.sqrt(diff_square.mean(axis=1))

In [ ]:
'''
10.46

beta_1 = 0.8397
beta_2 = 0.8028
decoder_dense_units = 19
epochs = 5
epsilon = 0.002189
learning_rate = 0.002168
lstm_units = 49
'''

In [ ]:
observations, predictions_lstm_v2 = moving_window_predictions(daily_data, model_name='lstm_v2', model_type='daily2weekly', lstm_units=49, decoder_dense_units=19, 
                                                              epochs=5, statistical_operation=statistical_operation, learning_rate=0.002168, beta_1=0.8397, 
                                                              beta_2=0.8028, epsilon=0.002189, n_input=n_input, n_out=n_out, n_gap=n_gap)

print(f'mse = {mean_squared_error(observations, predictions_lstm_v2)}')

diff = observations - predictions_lstm_v2

diff_square = np.power(diff, 2)
lstm_v2 = np.sqrt(diff_square.mean(axis=1))

In [ ]:
'''
10.40

beta_1 = 0.6191
beta_2 = 0.8347
decoder_dense_units = 10
epochs = 5
epsilon = 0.002229
learning_rate = 0.001867
lstm_units = 74
'''

In [ ]:
observations, predictions_lstm_v3 = moving_window_predictions(daily_data, model_name='lstm_v3', model_type='daily2weekly', lstm_units=74, decoder_dense_units=10, 
                                                              epochs=5, statistical_operation=statistical_operation, learning_rate=0.001867, beta_1=0.6191, 
                                                              beta_2=0.8347, epsilon=0.002229, n_input=n_input, n_out=n_out, n_gap=n_gap)

print(f'mse = {mean_squared_error(observations, predictions_lstm_v3)}')

diff = observations - predictions_lstm_v3

diff_square = np.power(diff, 2)
lstm_v3 = np.sqrt(diff_square.mean(axis=1))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 12))

ax.plot(baseline_365, label='base_line_rmse')
ax.plot(lstm_v1, label='lstm_v1_rmse')
ax.plot(lstm_v2, label='lstm_v2_rmse')
ax.plot(lstm_v3, label='lstm_v3_rmse')

ax.set_xlabel('train/val rmse round', fontsize=18)
ax.set_ylabel('RMSE (100,000)', fontsize=18)

ax.legend()

# 8 weeks to 4 weeks exploration

In [ ]:
import pandas as pd

from train.training_process import training_process
from main.utils.utils import optimization_process
from main.settings import train_validation_config

n_input = 56
n_out = 4
n_gap = 7

In [ ]:
model_name = 'lstm_v2'
model_type = 'weekly2weekly'

df = raw_data_preparation()

df['A_diff'] = df['A'].diff()

daily_data = df[['A', 'C', 'G', 'A_diff']]

pbounds = {'epochs': (5, 20),
           'lstm_units': (48, 130),
           'decoder_dense_units': (8, 20),
           'learning_rate': (0.0001, 0.003),
           'beta_1': (0.5, 0.95),
           'beta_2': (0.7, 0.9999),
           'epsilon': (0.00001, 0.01)}


statistical_operation = OrderedDict()

statistical_operation[0] = ['sum']
statistical_operation[1] = ['sum']
statistical_operation[2] = ['sum']
statistical_operation[3] = ['sum']

training_process_fn = partial(training_process, daily_data=daily_data, model_name=model_name,
                              model_type=model_type, n_out=n_out, n_gap=n_gap, n_input=n_input,
                              statistical_operation=statistical_operation)

optimized_parameters = optimization_process(training_process_fn, pbounds, model_name=model_name, model_type=model_type)